[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1972bNHN_BBTEorvLxqTvdRS4nC3PWj7O#scrollTo=jTQTcDbj9uxr)

### TMDB Box Office Prediction

**[1. Data loading and overview](#heading--1)**



**[2. Exploratory Data Analysis](#heading--2)**

  * [2.1. belongs_to_collection column](#heading--2-1)
  * [2.2. genres column](#heading--2-2)

In [1]:
import numpy as np
import pandas as pd
import ast
from collections import Counter

import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.float_format', lambda x: '%.4f' % x)


### 1. Data loading and overview <a name="heading--1"/>


In [2]:
# Loading train and test data
train_raw = pd.read_csv('train.csv')
test_raw = pd.read_csv('test.csv')

# copying of original data
train = train_raw.copy()
test = test_raw.copy()

# Id from test set is necessary to submit the file on Kaggle
test_id = test['id']

# 'revenue' is the target variable
target = train['revenue']

In [3]:
train.head()

,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,runtime,spoken_languages,status,tagline,title,Keywords,cast,crew,revenue
0,1,"[{'id': 313576, 'name': 'Hot Tub Time Machine ...",14000000,"[{'id': 35, 'name': 'Comedy'}]",NaN,tt2637294,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.5754,/tQtWuwvMf0hCc2QR2tkolwl7c3c.jpg,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",2/20/15,93.0000,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Laws of Space and Time are About to be Vio...,Hot Tub Time Machine 2,"[{'id': 4379, 'name': 'time travel'}, {'id': 9...","[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",12314651
1,2,"[{'id': 107674, 'name': 'The Princess Diaries ...",40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0368933,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.2489,/w9Z7A0GHEhIp7etpj0vyKOeU1Wx.jpg,"[{'name': 'Walt Disney Pictures', 'id': 2}]","[{'iso_3166_1': 'US', 'name': 'United States o...",8/6/04,113.0000,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It can take a lifetime to find true love; she'...,The Princess Diaries 2: Royal Engagement,"[{'id': 2505, 'name': 'coronation'}, {'id': 42...","[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",95149435
2,3,NaN,3300000,"[{'id': 18, 'name': 'Drama'}]",http://sonyclassics.com/whiplash/,tt2582802,en,Whiplash,"Under the direction of a ruthless instructor, ...",64.3000,/lIv1QinFqz4dlp5U4lQ6HaiskOZ.jpg,"[{'name': 'Bold Films', 'id': 2266}, {'name': ...","[{'iso_3166_1': 'US', 'name': 'United States o...",10/10/14,105.0000,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The road to greatness can take you to the edge.,Whiplash,"[{'id': 1416, 'name': 'jazz'}, {'id': 1523, 'n...","[{'cast_id': 5, 'character': 'Andrew Neimann',...","[{'credit_id': '54d5356ec3a3683ba0000039', 'de...",13092000
3,4,NaN,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",http://kahaanithefilm.com/,tt1821480,hi,Kahaani,Vidya Bagchi (Vidya Balan) arrives in Kolkata ...,3.1749,/aTXRaPrWSinhcmCrcfJK17urp3F.jpg,NaN,"[{'iso_3166_1': 'IN', 'name': 'India'}]",3/9/12,122.0000,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Kahaani,"[{'id': 10092, 'name': 'mystery'}, {'id': 1054...","[{'cast_id': 1, 'character': 'Vidya Bagchi', '...","[{'credit_id': '52fe48779251416c9108d6eb', 'de...",16000000
4,5,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,tt1380152,ko,마린보이,Marine Boy is the story of a former national s...,1.1481,/m22s7zvkVFDU9ir56PiiqIEWFdT.jpg,NaN,"[{'iso_3166_1': 'KR', 'name': 'South Korea'}]",2/5/09,118.0000,"[{'iso_639_1': 'ko', 'name': '한국어/조선말'}]",Released,NaN,Marine Boy,NaN,"[{'cast_id': 3, 'character': 'Chun-soo', 'cred...","[{'credit_id': '52fe464b9251416c75073b43', 'de...",3923970


In [4]:
test.head()

,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,runtime,spoken_languages,status,tagline,title,Keywords,cast,crew
0,3001,"[{'id': 34055, 'name': 'Pokémon Collection', '...",0,"[{'id': 12, 'name': 'Adventure'}, {'id': 16, '...",http://www.pokemon.com/us/movies/movie-pokemon...,tt1226251,ja,ディアルガVSパルキアVSダークライ,Ash and friends (this time accompanied by newc...,3.8515,/tnftmLMemPLduW6MRyZE0ZUD19z.jpg,NaN,"[{'iso_3166_1': 'JP', 'name': 'Japan'}, {'iso_...",7/14/07,90.0000,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Somewhere Between Time & Space... A Legend Is ...,Pokémon: The Rise of Darkrai,"[{'id': 11451, 'name': 'pok√©mon'}, {'id': 115...","[{'cast_id': 3, 'character': 'Tonio', 'credit_...","[{'credit_id': '52fe44e7c3a368484e03d683', 'de..."
1,3002,NaN,88000,"[{'id': 27, 'name': 'Horror'}, {'id': 878, 'na...",NaN,tt0051380,en,Attack of the 50 Foot Woman,When an abused wife grows to giant size becaus...,3.5598,/9MgBNBqlH1sG4yG2u4XkwI5CoJa.jpg,"[{'name': 'Woolner Brothers Pictures Inc.', 'i...","[{'iso_3166_1': 'US', 'name': 'United States o...",5/19/58,65.0000,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A titanic beauty spreads a macabre wave of hor...,Attack of the 50 Foot Woman,"[{'id': 9748, 'name': 'revenge'}, {'id': 9951,...","[{'cast_id': 2, 'character': 'Nancy Fowler Arc...","[{'credit_id': '55807805c3a3685b1300060b', 'de..."
2,3003,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",NaN,tt0118556,en,Addicted to Love,Good-natured astronomer Sam is devastated when...,8.0852,/ed6nD7h9sbojSWY2qrnDcSvDFko.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",5/23/97,100.0000,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A Comedy About Lost Loves And Last Laughs,Addicted to Love,"[{'id': 931, 'name': 'jealousy'}, {'id': 9673,...","[{'cast_id': 11, 'character': 'Maggie', 'credi...","[{'credit_id': '52fe4330c3a36847f8041367', 'de..."
3,3004,NaN,6800000,"[{'id': 18, 'name': 'Drama'}, {'id': 10752, 'n...",http://www.sonyclassics.com/incendies/,tt1255953,fr,Incendies,A mother's last wishes send twins Jeanne and S...,8.5960,/sEUG3qjxwHjxkzuO7plrRHhOZUH.jpg,"[{'name': 'TS Productions', 'id': 313}, {'name...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",9/4/10,130.0000,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,The search began at the opening of their mothe...,Incendies,"[{'id': 378, 'name': 'prison'}, {'id': 539, 'n...","[{'cast_id': 6, 'character': 'Nawal', 'credit_...","[{'credit_id': '56478092c3a36826140043af', 'de..."
4,3005,NaN,2000000,"[{'id': 36, 'name': 'History'}, {'id': 99, 'na...",NaN,tt0418753,en,Inside Deep Throat,"In 1972, a seemingly typical shoestring budget...",3.2177,/n4WC3zbelz6SG7rhkWbf8m9pMHB.jpg,NaN,"[{'iso_3166_1': 'US', 'name': 'United States o...",2/11/05,92.0000,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It was filmed in 6 days for 25 thousand dollar...,Inside Deep Throat,"[{'id': 279, 'name': 'usa'}, {'id': 1228, 'nam...","[{'cast_id': 1, 'character': 'Narrator (voice)...","[{'credit_id': '52fe44ce9251416c75041967', 'de..."


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     3000 non-null   int64  
 1   belongs_to_collection  604 non-null    object 
 2   budget                 3000 non-null   int64  
 3   genres                 2993 non-null   object 
 4   homepage               946 non-null    object 
 5   imdb_id                3000 non-null   object 
 6   original_language      3000 non-null   object 
 7   original_title         3000 non-null   object 
 8   overview               2992 non-null   object 
 9   popularity             3000 non-null   float64
 10  poster_path            2999 non-null   object 
 11  production_companies   2844 non-null   object 
 12  production_countries   2945 non-null   object 
 13  release_date           3000 non-null   object 
 14  runtime                2998 non-null   float64
 15  spok

In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4398 entries, 0 to 4397
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     4398 non-null   int64  
 1   belongs_to_collection  877 non-null    object 
 2   budget                 4398 non-null   int64  
 3   genres                 4382 non-null   object 
 4   homepage               1420 non-null   object 
 5   imdb_id                4398 non-null   object 
 6   original_language      4398 non-null   object 
 7   original_title         4398 non-null   object 
 8   overview               4384 non-null   object 
 9   popularity             4398 non-null   float64
 10  poster_path            4397 non-null   object 
 11  production_companies   4140 non-null   object 
 12  production_countries   4296 non-null   object 
 13  release_date           4397 non-null   object 
 14  runtime                4394 non-null   float64
 15  spok

In [7]:
# some basic statistics for numerical columns
train.describe().T

,count,mean,std,min,25%,50%,75%,max
id,3000.0000,1500.5000,866.1697,1.0000,750.7500,1500.5000,2250.2500,3000.0000
budget,3000.0000,22531334.1100,37026086.4120,0.0000,0.0000,8000000.0000,29000000.0000,380000000.0000
popularity,3000.0000,8.4633,12.1040,0.0000,4.0181,7.3749,10.8910,294.3370
runtime,2998.0000,107.8566,22.0864,0.0000,94.0000,104.0000,118.0000,338.0000
revenue,3000.0000,66725851.8887,137532326.3360,1.0000,2379808.2500,16807068.0000,68919203.5000,1519557910.0000


In [8]:
test.describe().T

,count,mean,std,min,25%,50%,75%,max
id,4398.0000,5199.5000,1269.7376,3001.0000,4100.2500,5199.5000,6298.7500,7398.0000
budget,4398.0000,22649291.1173,36899910.8813,0.0000,0.0000,7450000.0000,28000000.0000,260000000.0000
popularity,4398.0000,8.5502,12.2090,0.0000,3.8952,7.4822,10.9385,547.4883
runtime,4394.0000,107.6222,21.0583,0.0000,94.0000,104.0000,118.0000,320.0000


In [9]:
# some basic statistics for object columns
train.describe(include=['object']).T

,count,unique,top,freq
belongs_to_collection,604,422,"[{'id': 645, 'name': 'James Bond Collection', ...",16
genres,2993,872,"[{'id': 18, 'name': 'Drama'}]",266
homepage,946,941,http://www.transformersmovie.com/,4
imdb_id,3000,3000,tt0046126,1
original_language,3000,36,en,2575
original_title,3000,2975,Red Dawn,2
overview,2992,2992,Missy McCloud is the most beautiful girl in sc...,1
poster_path,2999,2999,/1JDfbdR6RaLc8smQRZmDadQrmhw.jpg,1
production_companies,2844,2383,"[{'name': 'Paramount Pictures', 'id': 4}]",51
production_countries,2945,321,"[{'iso_3166_1': 'US', 'name': 'United States o...",1752


In [10]:
test.describe(include=['object']).T

,count,unique,top,freq
belongs_to_collection,877,556,"[{'id': 645, 'name': 'James Bond Collection', ...",10
genres,4382,1101,"[{'id': 18, 'name': 'Drama'}]",348
homepage,1420,1402,http://www.kungfupanda.com/,3
imdb_id,4398,4398,tt0049408,1
original_language,4398,39,en,3776
original_title,4398,4353,Psycho,2
overview,4384,4383,No overview found.,2
poster_path,4397,4397,/lh6EZuSSmGzaRgYuJeO7uSynQR4.jpg,1
production_companies,4140,3366,"[{'name': 'Paramount Pictures', 'id': 4}]",72
production_countries,4296,458,"[{'iso_3166_1': 'US', 'name': 'United States o...",2587


In [11]:
print(f'the shape of the train set: {train.shape}')
print(f'the shape of the test set: {test.shape}')

the shape of the train set: (3000, 23)
the shape of the test set: (4398, 22)


### 2. Exploratory Data Analysis <a name="heading--2"/>

There are JSON format columns so first the should be converted to python dictionary format.

In [12]:
json_cols = ['belongs_to_collection', 'genres', 'production_companies',
            'production_countries', 'spoken_languages', 'Keywords', 
            'cast', 'crew']

def json_to_dict(df):
    for col in json_cols:
        df[col] = df[col].apply(lambda x: {} if pd.isna(x) else ast.literal_eval(x))
    return df

train = json_to_dict(train)
test = json_to_dict(test)

#### 2.1. belongs_to_collection column <a name="heading--2-1"/>

In [13]:
train['belongs_to_collection'][0]

[{'backdrop_path': '/noeTVcgpBiD48fDjFVic1Vz7ope.jpg',
  'id': 313576,
  'name': 'Hot Tub Time Machine Collection',
  'poster_path': '/iEhb00TGPucF0b4joM1ieyY026U.jpg'}]

In [14]:
# checking how many rows in 'belongs_to_collection' column is empty
train['belongs_to_collection'].apply(lambda x: len(x) if x != {} else 0).value_counts()


0    2396
1     604
Name: belongs_to_collection, dtype: int64

In [15]:
# 25 most frequent collection name without empty values
train['belongs_to_collection'].apply(lambda x: x[0]['name'] if x != {} else 0).value_counts()[1:25]

James Bond Collection                        16
Friday the 13th Collection                    7
The Pink Panther (Original) Collection        6
Police Academy Collection                     5
Pokémon Collection                            5
Rocky Collection                              4
Resident Evil Collection                      4
Rambo Collection                              4
Transformers Collection                       4
Child's Play Collection                       4
The Fast and the Furious Collection           4
Paranormal Activity Collection                4
Alien Collection                              4
Ice Age Collection                            4
Rush Hour Collection                          3
The Wolverine Collection                      3
The Dark Knight Collection                    3
Cars Collection                               3
Halloween Collection                          3
Star Trek: The Original Series Collection     3
Qatsi Collection                        

I think only collection name is important. The others can be deleted. Because 'collection_name' is a new column so 'belongs_to_collection' column can be also deleted

In [16]:
train['collection_name'] = train['belongs_to_collection'].apply(lambda x: x[0]['name'] if x != {} else 0)
test['collection_name'] = test['belongs_to_collection'].apply(lambda x: x[0]['name'] if x != {} else 0)

train = train.drop(['belongs_to_collection'], axis=1)
test = test.drop(['belongs_to_collection'], axis=1)

#### 2.2. genres column <a name="heading--2-2"/>

In [17]:
for i, e in enumerate(train['genres'][:5]):
    print(i, e)

0 [{'id': 35, 'name': 'Comedy'}]
1 [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10751, 'name': 'Family'}, {'id': 10749, 'name': 'Romance'}]
2 [{'id': 18, 'name': 'Drama'}]
3 [{'id': 53, 'name': 'Thriller'}, {'id': 18, 'name': 'Drama'}]
4 [{'id': 28, 'name': 'Action'}, {'id': 53, 'name': 'Thriller'}]


In [18]:
train['movie_genres'] = train['genres'].apply(lambda x: [i['name'] for i in x] if x!={} else [])
train['movie_genres']

0                                 [Comedy]
1         [Comedy, Drama, Family, Romance]
2                                  [Drama]
3                        [Thriller, Drama]
4                       [Action, Thriller]
                       ...                
2995                     [Comedy, Romance]
2996                        [Drama, Music]
2997    [Crime, Action, Mystery, Thriller]
2998                     [Comedy, Romance]
2999           [Thriller, Action, Mystery]
Name: movie_genres, Length: 3000, dtype: object

In [21]:
test['movie_genres'] = test['genres'].apply(lambda x: [i['name'] for i in x] if x!={} else [])
test['movie_genres']

0       [Adventure, Animation, Family, Fantasy]
1                     [Horror, Science Fiction]
2                             [Comedy, Romance]
3                         [Drama, War, Mystery]
4                        [History, Documentary]
                         ...                   
4393                                 [Thriller]
4394      [Action, Adventure, Comedy, Thriller]
4395                                    [Drama]
4396                         [Horror, Thriller]
4397                                    [Drama]
Name: movie_genres, Length: 4398, dtype: object

In [22]:
train['genres_number'] = train['movie_genres'].apply(len)
train[['title', 'movie_genres', 'genres_number']][:5]

,title,movie_genres,genres_number
0,Hot Tub Time Machine 2,[Comedy],1
1,The Princess Diaries 2: Royal Engagement,"[Comedy, Drama, Family, Romance]",4
2,Whiplash,[Drama],1
3,Kahaani,"[Thriller, Drama]",2
4,Marine Boy,"[Action, Thriller]",2


In [23]:
test['genres_number'] = test['movie_genres'].apply(len)
test[['title', 'movie_genres', 'genres_number']][:5]

,title,movie_genres,genres_number
0,Pokémon: The Rise of Darkrai,"[Adventure, Animation, Family, Fantasy]",4
1,Attack of the 50 Foot Woman,"[Horror, Science Fiction]",2
2,Addicted to Love,"[Comedy, Romance]",2
3,Incendies,"[Drama, War, Mystery]",3
4,Inside Deep Throat,"[History, Documentary]",2


unique value of genres in train set

In [24]:
train_li = []
[train_li.append(dic['name']) for li in train['genres'] for dic in li if dic['name'] not in train_li]
sorted(train_li)

['Action',
 'Adventure',
 'Animation',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Foreign',
 'History',
 'Horror',
 'Music',
 'Mystery',
 'Romance',
 'Science Fiction',
 'TV Movie',
 'Thriller',
 'War',
 'Western']

unique value of genres in test set

In [25]:
test_li = []
[test_li.append(dic['name']) for li in test['genres'] for dic in li if dic['name'] not in test_li]
sorted(test_li)

['Action',
 'Adventure',
 'Animation',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Foreign',
 'History',
 'Horror',
 'Music',
 'Mystery',
 'Romance',
 'Science Fiction',
 'Thriller',
 'War',
 'Western']

In [27]:

genres_count = Counter([i for j in train['movie_genres'] for i in j]).most_common()
genres_count

[('Drama', 1531),
 ('Comedy', 1028),
 ('Thriller', 789),
 ('Action', 741),
 ('Romance', 571),
 ('Crime', 469),
 ('Adventure', 439),
 ('Horror', 301),
 ('Science Fiction', 290),
 ('Family', 260),
 ('Fantasy', 232),
 ('Mystery', 225),
 ('Animation', 141),
 ('History', 132),
 ('Music', 100),
 ('War', 100),
 ('Documentary', 87),
 ('Western', 43),
 ('Foreign', 31),
 ('TV Movie', 1)]